# Global Degiro Account PnL

## Importing the Transactions

In [ ]:
import pandas as pd

trx_df = pd.read_csv(
    "data/Transactions.csv",
    encoding="utf-8-sig",
    sep=",",
    usecols=["Produit", "Code ISIN", "Quantité", "Montant négocié"],
)
trx_df.rename(columns={"Produit": "Product", "Code ISIN": "ISIN", "Quantité": "Quantity", "Montant négocié": "Negociated amount"}, inplace=True)

trx_df.head()

## Getting Market Prices and Fx Rates

- The Porfolio CSV gives the market price of the positions at the moment of the download
- Some positions are in USD. But the csv gives also the EUR amount, which makes it possible to compute the rate.

In [ ]:
portfolio_df = pd.read_csv(
    "data/Portfolio.csv",
    encoding="utf-8-sig",
    sep=",",
    usecols=["Ticker/ISIN", "Clôture", "Devise", "Montant en EUR"],
)
portfolio_df["Devise"] = portfolio_df["Devise"].str.split(" ").str[1]
portfolio_df.rename(columns={"Ticker/ISIN": "ISIN", "Clôture": "Market price", "Devise": "Amount in Original Currency", "Montant en EUR": "Amount in EUR"}, inplace=True)

# Rows with NA are not needed, let's drop them
portfolio_df = portfolio_df.dropna()

# Convert to float
portfolio_df["Market price"] = portfolio_df["Market price"].str.replace(",", ".", regex=False)
portfolio_df["Market price"] = portfolio_df["Market price"].astype(float)
portfolio_df["Amount in EUR"] = portfolio_df["Amount in EUR"].str.replace(",", ".", regex=False)
portfolio_df["Amount in EUR"] = portfolio_df["Amount in EUR"].astype(float)
portfolio_df["Amount in Original Currency"] = portfolio_df["Amount in Original Currency"].astype(float)

portfolio_df.head()

In [ ]:
portfolio_df["fx_rate"] = portfolio_df["Amount in EUR"] / portfolio_df["Amount in Original Currency"]
portfolio_df.head()

## Enriching the Transactions with Market Price and Fx Rates

In [ ]:
# We don't need these columns anymore as fx_rate as been calculated
portfolio_df.drop(columns=["Amount in Original Currency", "Amount in EUR"], inplace=True)

df = pd.merge(trx_df, portfolio_df, left_on="ISIN", right_on="ISIN", how="left")

df.head()

## Grouping Transactions by Product

In [ ]:
lines = df.groupby(["Product", "ISIN"]).agg({
    "Quantity": "sum",
    "Negociated amount": "sum",
    "Market price": "first",
    "fx_rate": "first"
})

lines

## Getting the Dividends and Taxes on Dividends

Problem: some dividends are in USD, and it's not possible to compute the FX rate with the data from the CSV files.

In [ ]:
account_df = pd.read_csv(
    "data/Account.csv",
    encoding="utf-8-sig",
    sep=",",
    usecols=["Code ISIN", "Description", "Mouvements", "Unnamed: 8"],
)
account_df.rename(columns={"Code ISIN": "ISIN", "Description": "Kind", "Mouvements": "Currency", "Unnamed: 8": "Amount"}, inplace=True)

dividend_df = account_df[account_df["Kind"].isin(["Dividende"])]
dividend_df = dividend_df.groupby(["ISIN", "Currency"]).agg({"Amount": "sum"})
dividend_df.reset_index(inplace=True)
dividend_df.set_index("ISIN", inplace=True)
dividend_df.head()

In [ ]:
tax_df = account_df[account_df["Kind"] == "Impôts sur dividende"]
tax_df = tax_df.groupby(["ISIN", "Currency"]).agg({"Amount": "sum"})
tax_df.reset_index(inplace=True)
tax_df.set_index("ISIN", inplace=True)
tax_df

## Computing the PnL for each Product

For the closed positions, market price is NaN. In this cases, "Negociated Amount" is the realized PnL.

When market price is not NaN, it means some positions are open for this product.
Therefore, "Negociated Amount" is the cost basis.  
But some positions might have been closed already for this product.
In this situation "Negociated Amount" might contains some realized PnL as well.

In [ ]:
lines.fillna({"Market price": 0, "fx_rate": 0 }, inplace=True)

lines.reset_index(inplace=True)
lines.set_index("ISIN", inplace=True)

lines = pd.merge(lines,tax_df[["Amount"]], on="ISIN", how="left")
lines.rename(columns={"Amount": "Tax"}, inplace=True)
lines["Tax"] = lines["Tax"].fillna(0)

lines = pd.merge(lines,dividend_df[["Amount", "Currency"]], on="ISIN", how="left")
lines.rename(columns={"Amount": "Dividend", "Currency": "Dividend Currency"}, inplace=True)
lines["Dividend"] = lines["Dividend"].fillna(0)
lines["Dividend Currency"] = lines["Dividend Currency"].fillna("EUR")

lines["Net Dividend"] = lines["Dividend"] + lines["Tax"]
lines.drop(columns=["Tax", "Dividend"], inplace=True)

lines["PnL"] = lines["Negociated amount"] + (lines["Quantity"] * lines["Market price"] * lines["fx_rate"])

pnl_without_dividends = lines["PnL"].sum()

pnl_lines = lines.copy()

# Build as Str to mix USD dividends and EUR PnL
pnl_lines["PnL"] = pnl_lines.apply(
    lambda row: f"{row['PnL']:.2f} EUR" + 
    (f" + {row['Net Dividend']:.2f} {row['Dividend Currency']}" if row['Net Dividend'] != 0 else ""),
    axis=1
)

# Cleaning
pnl_lines["Has Open Positions"] = pnl_lines["Quantity"].gt(0)
pnl_lines.drop(columns=["Net Dividend", "Dividend Currency", "fx_rate", "Market price", "Negociated amount", "Quantity"], inplace=True)



pnl_lines

In [ ]:
net_eur_dividends = dividend_df[dividend_df["Currency"] == "EUR"]["Amount"].sum() + tax_df[tax_df["Currency"] == "EUR"]["Amount"].sum()
net_usd_dividends = dividend_df[dividend_df["Currency"] == "USD"]["Amount"].sum() + tax_df[tax_df["Currency"] == "USD"]["Amount"].sum()

net_eur_dividends
net_usd_dividends

## Computing Total Deposit on the Account 

In [ ]:
accounts_df = pd.read_csv(
    "data/Account.csv",
    encoding="utf-8-sig",
    sep=",",
    usecols=["Description", "Mouvements", "Unnamed: 8"],
)

deposits_df = accounts_df[accounts_df["Description"] == "Dépôt flatex"].copy()
deposits_df.rename(columns={"Mouvements": "Currency", "Unnamed: 8": "Amount"}, inplace=True)
total_deposits = deposits_df.Amount.sum()

# I have no withdrawals to take into account, only deposits

total_deposits

## Finding Account Balance

In [ ]:
portfolio_df = pd.read_csv(
    "data/Portfolio.csv",
    encoding="utf-8-sig",
    sep=",",
    usecols=["Produit", "Montant en EUR"],
)
portfolio_df = portfolio_df[portfolio_df["Produit"].str.contains("CASH & CASH FUND & FTX CASH")]
portfolio_df["Montant en EUR"] = portfolio_df["Montant en EUR"].str.replace(",", ".", regex=False)
portfolio_df["Montant en EUR"] = portfolio_df["Montant en EUR"].astype(float)
account_balance = portfolio_df["Montant en EUR"].sum()

account_balance


## Computing Account Total Valuation

In [ ]:
account_valuation = (lines["Quantity"] * lines["Market price"] * lines["fx_rate"]).sum() + account_balance
account_valuation

## Account Global Analysis

In [ ]:
print(f"Total Deposits: €{total_deposits:,.2f}")
print(f"Current Portfolio Value: €{account_valuation:,.2f}")
print(f"Portfolio Performance: €{account_valuation - total_deposits:,.2f}")

print(f"Total PnL (without dividends): €{pnl_without_dividends:,.2f}")
print(f"Total EUR Dividends: €{net_eur_dividends:,.2f}")
print(f"Total USD Dividends: ${net_usd_dividends:,.2f}")